### Часть 2. Тематическое моделирование
1. Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
2. Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?
Эта часть задания может быть сделана с использованием gensim.

In [1]:
import json

import bz2
import re

from tqdm import tqdm
import pandas as pd
%matplotlib inline
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Загружаем данные и готовим DF

In [2]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:02, 3228.78it/s]


In [3]:
df = pd.DataFrame(responses)

Функции для подготовки текста для последующего анализа
- стоп слова
- лемматизация
- токенизация через регулярку (только слова из русских букв) и последующий split

In [4]:
from string import punctuation
from nltk.corpus import stopwords

stops  = stopwords.words('russian') + list(punctuation) + ["``", "''"] + [
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
]
ru_words = re.compile("[А-Яа-я]+")

In [5]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

In [6]:
def words_only(text):
    return " ".join(ru_words.findall(text))

# def lemmatize(text, mystem=m):
#     try:
#         return "".join(m.lemmatize(text)).strip()
#     except:
#         return " "

def lemmatize(text, mystem=morph):
    try:
        return " ".join([mystem.parse(w)[0].normal_form for w in text.split(' ')]).strip()
    except:
        return " "

def remove_stopwords(text, mystopwords = stops):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

In [9]:
%%time
df_analyze = df[:1000].copy()
df_analyze.text = df_analyze.text.apply(preprocess)

CPU times: total: 41.4 s
Wall time: 41.3 s


Создаем и тренируем модели с разным количеством тем (5, 15, 30)

In [10]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, lsimodel
from gensim import similarities

In [20]:
texts = [text.split() for text in df_analyze.text]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [21]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [22]:
n_topics = [5, 15, 30]
lsi_models = {}
for n in n_topics:
    lsi_models[n] = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=n)

In [26]:
lsi_models[30].show_topics(30)

[(0,
  '0.194*"карта" + 0.152*"вклад" + 0.145*"кредит" + 0.113*"деньга" + 0.107*"сумма" + 0.105*"счёт" + 0.096*"договор" + 0.096*"сотрудник" + 0.095*"отделение" + 0.094*"заявление"'),
 (1,
  '-0.572*"вклад" + -0.298*"ставка" + -0.263*"пополнение" + -0.205*"условие" + -0.123*"односторонний" + -0.117*"изменение" + -0.115*"вкладчик" + -0.114*"договор" + 0.104*"кредит" + 0.104*"звонок"'),
 (2,
  '-0.245*"звонок" + 0.234*"карта" + -0.217*"номер" + -0.166*"телефон" + -0.156*"данные" + -0.156*"ваш" + -0.155*"согласие" + -0.149*"персональный" + -0.144*"удалить" + -0.132*"давать"'),
 (3,
  '0.421*"страховка" + 0.358*"страхование" + 0.332*"кредит" + -0.188*"карта" + 0.150*"договор" + -0.127*"банкомат" + -0.124*"номер" + 0.113*"отказаться" + 0.112*"досрочный" + 0.106*"отказ"'),
 (4,
  '0.209*"задолженность" + 0.174*"сумма" + 0.165*"платёж" + 0.160*"средство" + 0.143*"погашение" + 0.132*"денежный" + 0.130*"руб" + -0.129*"отделение" + -0.129*"офис" + 0.122*"комиссия"'),
 (5,
  '0.242*"страхование" 

Пример темы, в которой можно понять смысл сообщений:
(4,
  '0.209*"задолженность" + 0.174*"сумма" + 0.165*"платёж" + 0.160*"средство" + 0.143*"погашение" + 0.132*"денежный" + 0.130*"руб" + -0.129*"отделение" + -0.129*"офис" + 0.122*"комиссия"'),

Вероятно, речь идет о возможности погасить задолженность в отделении банка

In [50]:
for t in range(lsi_models[30].num_topics):
    if "сбербанк" in lsi_models[30].print_topic(t):
        print(t, ":\n",  lsi_models[30].show_topic(t))

5 :
 [('страхование', 0.24202996972745808), ('страховка', 0.24022042711689728), ('сбербанк', 0.22165165508964155), ('вернуть', 0.1450802803665642), ('денежный', 0.13572301578470827), ('средство', 0.1347369219794901), ('платёж', -0.12941492068460017), ('г', 0.12835119588206267), ('просрочка', -0.12532210679099992), ('возврат', 0.11901467127635443)]
6 :
 [('карта', -0.330924789982194), ('банкомат', 0.22258735768569995), ('р', -0.2138724454167469), ('сбербанк', 0.21258679259677565), ('комиссия', -0.18144332625854226), ('лимит', -0.17233564304357543), ('страховка', -0.1345903648677649), ('денежный', 0.134366230567053), ('средство', 0.12065006362415427), ('погашение', 0.10617427340972226)]
8 :
 [('карта', -0.2516947907897725), ('сумма', 0.15557570018855638), ('лимит', -0.1554308432199808), ('кредитный', -0.1517421876208719), ('р', 0.14354934996731883), ('деньга', 0.1435057243688443), ('счёт', 0.13948132859324103), ('банкомат', -0.13355482349444422), ('сбербанк', -0.13130699945257177), ('пла

При разбиении на 30 тем, можно найти ряд тем содержащих "сбербанк"
Некоторые из них можно достаточно легко пркомментировать.
Например, тема 6 явно связана с погашением кредита/креитной карты